In [ ]:
!git install llmware
!pip install faiss-cpu
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 25.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvid

In [ ]:
import os
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.status import Status
from llmware.prompts import Prompt
from llmware.configs import LLMWareConfig
from importlib import util
if not util.find_spec("chromadb"):
  print("\nto run this example with chromadb python sdk: pip3 install chromadb")

In [ ]:
LLMWareConfig().set_active_db("sqlite")

In [ ]:
embedding_model = "industry-bert-contracts"

In [ ]:
vector_db = "chomadb"

In [ ]:
lib_name = "example_5_library"

In [ ]:
example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]
llm_model_name = example_models[0]

In [ ]:
print("\nupdate: Step 1 - Creating library: {}".format(lib_name))
library = Library().create_new_library(lib_name)


update: Step 1 - Creating library: example_5_library


In [ ]:
print("update: Step 2 - Downloading Sample Files")
sample_files_path = Setup().load_sample_files(over_write=False)
contracts_path = os.path.join(sample_files_path, "Agreements")

update: Step 2 - Downloading Sample Files


In [ ]:
print("update: Step 3 - Parsing and Text Indexing Files")
library.add_files(input_folder_path=contracts_path, chunk_size=400, max_chunk_size=600, smart_chunking=1)

update: Step 3 - Parsing and Text Indexing Files


{'docs_added': 0,
 'blocks_added': 0,
 'images_added': 0,
 'pages_added': 0,
 'tables_added': 0,
 'rejected_files': []}

In [ ]:
print("\nupdate: Step 4 - Generating Embeddings in {} db - with Model- {}".format(vector_db, embedding_model))
library.install_new_embedding(embedding_model=embedding_model, vector_db=vector_db)


update: Step 4 - Generating Embeddings in chomadb db - with Model- industry-bert-contracts


TypeError: Library.install_new_embedding() got an unexpected keyword argument 'embedding_model'

In [ ]:
print("\nupdate: Loading model for LLM inference - ", llm_model_name)
prompter = Prompt().load_model(llm_model_name)
query = "what is the executive's base annual salary"
results = Query(library).semantic_query(query, result_count=50, embedding_distance_threshold=1.0)

In [ ]:
for i, contract in enumerate(os.listdir(contracts_path_)):

  qr = []

  if contract != ".DS_Store":
    print("\nContract Name: ", i, contract)

    for j, entries in enumerate(results):
      library_fn = entries["file_source"]

      if os.sep in library_fn:
        library_fn = library_fn.split(os.sep)[-1]

      if library_fn == contract:
        print("Top Retrieval: ", j, entries["distance"], entries["text"])
        qr.append(entries)

    source = prompter.add_source_query_results(query_results=qr)

    respose = prompter.prompt_with_source(query, prompt_name="default_with_context", temperature=0.3)

    for resp in response:
      if "llm_response" in resp:
        print("\nupdate: llm answer - ", resp["llm_response"])

    prompter.clear_source_materials()

  return 0